安装依赖（torch版本需要自己调整）

In [ ]:
!conda install pytorch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 pytorch-cuda=12.1 -c pytorch -c nvidia -y
!pip install trl modelscope transformers -i https://mirrors.aliyun.com/pypi/simple/ 

下载模型

In [ ]:
from modelscope import snapshot_download
model_dir = snapshot_download('Qwen/Qwen2.5-0.5B-Instruct')

ref和policy模型

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "C:\\Users\\owen\\.cache\\modelscope\\hub\\Qwen\\Qwen2___5-0___5B-Instruct"

policy = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
ref = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

policy生成答案

In [8]:
import torch 

def gen_completion(model,query,tokenizer):
    SYSTEM='''
    Respond in the following format:
    <reasoning>
    ...
    </reasoning>
    <answer>
    ...
    </answer>
    '''
    messages=[{'role':'system','content':SYSTEM},{'role':'user','content':query}]
    text = tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs
        )
    completion_ids=generated_ids[0][len(model_inputs.input_ids[0]):]
    completion_text=tokenizer.decode(completion_ids, skip_special_tokens=True)
    return model_inputs.input_ids[0],text,completion_ids,completion_text

gen_completion(policy,'3+5等于几呢？',tokenizer)

(tensor([151644,   8948,    271,    262,  39533,    304,    279,   2701,   3561,
            510,    262,    366,  19895,    287,    397,    262,  12236,    262,
            690,  19895,    287,    397,    262,    366,   9217,    397,    262,
          12236,    262,    690,   9217,    397,    257, 151645,    198, 151644,
            872,    198,     18,     10,     20, 107106,  99195, 101036,  11319,
         151645,    198, 151644,  77091,    198], device='cuda:0'),
 '<|im_start|>system\n\n    Respond in the following format:\n    <reasoning>\n    ...\n    </reasoning>\n    <answer>\n    ...\n    </answer>\n    <|im_end|>\n<|im_start|>user\n3+5等于几呢？<|im_end|>\n<|im_start|>assistant\n',
 tensor([    18,    488,    220,     20,    284,    220,     23, 151645],
        device='cuda:0'),
 '3 + 5 = 8')

计算model预测的目标token概率

In [73]:
input_ids,input_text,completion_ids,completion_text=gen_completion(policy,'3+5等于几呢？',tokenizer)

def gen_probs(model,input_ids,completion_ids):
    full_ids=torch.cat([input_ids,completion_ids],dim=0).unsqueeze(0)
    output=model(full_ids)
    probs=torch.nn.functional.softmax(output.logits,dim=-1)
    prob_select=full_ids[:,1:].unsqueeze(-1)
    token_probs=torch.gather(probs[:,:-1],dim=-1,index=prob_select)[0]
    return token_probs[len(input_ids)-1:,0]

ref_token_probs=gen_probs(ref,input_ids,completion_ids)
print(ref_token_probs.shape,completion_ids.shape,completion_text)

torch.Size([11]) torch.Size([11]) 3 + 5 等于8。
